In [165]:
import pandas as pd
import numpy as np
from datetime import timedelta, date, datetime
import warnings
warnings.filterwarnings('ignore')

In [180]:
cols_to_use = ['HubRandomID', 'Hub_ActivityID', 'ActivityDate', 
               'ProgrammeID', 'ChildParticipants', 'AdultParticipants', 'ReferralParticipants', 
               'EngagedCount', 'StartTime', 'EndTime', 'HubVolunteers',
              'SchoolVolunteers', 'ExternalVolunteers', 'ProgrammeID', 
              'ProgrammeID.1', 'ServiceTypeID', 'ProgrammeCategoryID',
              'ProgrammeCategoryID.1', 'ShortName.1', 'CategoryName', 'ProgrammeName']

col_types = {'HubRandomID': 'int64', 'Hub_ActivityID': 'int64', 'ActivityDate': 'str', 
             'ProgrammeID': 'int64', 'ChildParticipants': 'int64', 'AdultParticipants': 'int64', 
             'ReferralParticipants': 'int64', 'EngagedCount': 'int64', 'StartTime': 'str', 
             'EndTime' : 'str', 'HubVolunteers': 'int64','SchoolVolunteers': 'int64', 
             'ExternalVolunteers': 'int64', 'ProgrammeID': 'int64', 'ProgrammeID.1': 'int64', 
             'ServiceTypeID': 'int64', 'ProgrammeCategoryID': 'int64', 'ProgrammeCategoryID.1': 'int64', 
             'ShortName.1': 'str', 'CategoryName': 'str', 'ProgrammeName': 'str'}

datetime_cols = ['ActivityDate', 'StartTime', 'EndTime']

df = pd.read_csv('Datathon - CHA activities 2016-2020.csv', usecols=cols_to_use, dtype=col_types, parse_dates=datetime_cols).sort_values(by="ActivityDate")

In [181]:
df.tail()

,HubRandomID,Hub_ActivityID,ActivityDate,ProgrammeID,ChildParticipants,AdultParticipants,ReferralParticipants,EngagedCount,StartTime,EndTime,ExternalVolunteers,HubVolunteers,SchoolVolunteers,ProgrammeID.1,ProgrammeName,ProgrammeCategoryID,ServiceTypeID,ProgrammeCategoryID.1,CategoryName,ShortName.1
157586,13590,659379,2019-12-20,55,0,0,0,0,2020-02-28 09:00:00,2020-02-28 15:00:00,0,0,0,55,Volunteering,2,2,2,Education and Training,VET
158333,11818,660493,2019-12-20,52,0,0,0,0,2020-02-28 11:00:00,2020-02-28 14:00:00,0,0,0,52,Formal Training,2,3,2,Education and Training,VET
156745,11188,658144,2019-12-20,49,0,0,0,0,2020-02-28 08:30:00,2020-02-28 09:15:00,0,2,0,49,Parent engagement,1,10,1,Programmed Activities,PROG
162160,11188,667795,2019-12-20,52,0,15,0,0,2020-02-28 09:30:00,2020-02-28 14:30:00,0,0,0,52,Formal Training,2,3,2,Education and Training,VET
157243,17600,658968,2019-12-20,49,0,0,0,0,2020-02-28 12:00:00,2020-02-28 14:00:00,0,0,0,49,Parent engagement,1,10,1,Programmed Activities,PROG


### Descriptive Stats

#### General Hub Stats

In [182]:
# HubRandomID
# how many hubs?
df['HubRandomID'].nunique()

80

In [183]:
# how many hubs opened in the last year 
# LOGIC: is their first activity date within the last year 

# get a list of hub ids
hub_ids= list(df['HubRandomID'].unique())

# find the first instance of the id    
first_idx = [df['HubRandomID'].ne(x).idxmin() for x in hub_ids]   
df_fist_instance = df.iloc[first_id]

# get list of hubs that opened by year
min_date = df_fist_instance['ActivityDate'].min().year
max_date = df_fist_instance['ActivityDate'].max().year
date_range = list(range(min_date, max_date  + 1, 1))
[print(f"The number of hubs opened in {x}: {len(df_fist_instance.loc[df_fist_instance['ActivityDate'].dt.year == x])}")\
 for x in date_range]


The number of hubs opened in 2016: 45
The number of hubs opened in 2017: 13
The number of hubs opened in 2018: 11
The number of hubs opened in 2019: 11


[None, None, None, None]

#### Activity Stats

In [184]:
# when did they start recording times? 
df_valid_start_end_time = df[df['StartTime'].isnull() == False]
print(f"When did CHA started recording start/end times of activities? {df_valid_start_end_time['ActivityDate'].min()} ")

When did CHA started recording start/end times of activities? 2018-01-08 00:00:00 


In [185]:
# how many activities?
how_many_acts = df.groupby('HubRandomID')['Hub_ActivityID']\
.agg(['count'])

print(f"Hub with Max ({how_many_acts['count'].max()}) Activites: "
      f"{how_many_acts['count'].idxmax()} \n")
print(f"Hub with Min ({how_many_acts['count'].min()}) Activities: " 
      f"{how_many_acts['count'].idxmin()}")

Hub with Max (5280) Activites: 19277 

Hub with Min (37) Activities: 10932


In [186]:
# how long does a hub activity typically go for?

# create a column called total time 
df_valid_start_end_time['TotalActivityTime']= (df_valid_start_end_time['EndTime'] - df_valid_start_end_time['StartTime'])/np.timedelta64(1, 'm')

# average over that column 
print(f"A typical hub activity goes for {round(df_valid_start_end_time['TotalActivityTime'].mean(), 2)} minutes")

A typical hub activity goes for 95.92 minutes


In [265]:
def count_df(col:str, df:pd.DataFrame) -> pd.DataFrame:
    """
    get the number of times each value in a column occurs
    returned in a dataframe
    """
    occurance_name = 'NumberOfOccur'
    name_key = list(df[col].unique())
    name_values = [len(df.loc[df[col] == x]) for x in name_key]
    return pd.DataFrame.from_dict(dict(zip(name_key, name_values)), orient='index', columns=[occurance_name])\
.sort_values(by=occurance_name, ascending=False)

In [267]:
# what is the most common activity (by programe name) 
count_programme_name = count_df('ProgrammeName', df)
print(f"The most common ({count_programme_name['NumberOfOccur'].max():,.0f}) programme is {count_programme_name['NumberOfOccur'].idxmax()}")

The most common (27,936) programme is Parent engagement


In [269]:
# what is the most common activity (by category name)
count_programme_cat = count_df('CategoryName', df)
print(f"The most common ({count_programme_cat['NumberOfOccur'].max():,.0f}) programme category is {count_programme_cat['NumberOfOccur'].idxmax()}")

The most common (88,458) programme category is Programmed Activities


#### Participant Stats

In [173]:
# total participants column (sum of all the participants)
particpant_cols = ['ChildParticipants', 'AdultParticipants']
df['TotalParticipants'] = df[particpant_cols].sum(axis=1)

In [174]:
# average attendance per session per hub?
attendance = df.groupby('HubRandomID')['TotalParticipants']\
.agg(['sum', 'mean'])

print(f"Hub with the highest average attendance ({round(attendance['mean'].max(), 2)}): "
      f"{attendance['mean'].idxmax()} \n")
print(f"Hub with the lowest average attendance ({round(attendance['mean'].min(), 2)}): "
      f"{attendance['mean'].idxmin()}")

Hub with the highest average attendance (36.39): 18922 

Hub with the lowest average attendance (1.43): 10932


In [175]:
# who attends more: adults or children? 
total_particpants = df['TotalParticipants'].sum()
adult_participants = df['AdultParticipants'].sum()
child_participants = df['ChildParticipants'].sum()
print(f"Total number of sessions: {total_particpants:,.0f}\n")
print(f"Adults attend {adult_participants:,.0f} sessions; {round(adult_participants/total_particpants * 100, 2)}% of the total participants\n")
print(f"Children attend {child_participants:,.0f} sessions; {round(child_participants/total_particpants * 100, 2)}% of the total participants")

Total number of sessions: 1,594,194

Adults attend 617,140 sessions; 38.71% of the total participants

Children attend 977,054 sessions; 61.29% of the total participants


In [176]:
# what percent of people come by referral? 
referred_participants = df['ReferralParticipants'].sum()
print(f"Total number of referred participants {referred_participants:,.0f}; {round(referred_participants/total_particpants * 100, 2)}% of the total participants")

Total number of referred participants 164,159; 10.3% of the total participants


#### Volunteer Stats

In [177]:
# how many volunteers? 

# create a sum of volunteers
df['TotalVolunteerNumber'] = df['ExternalVolunteers'] + df['HubVolunteers']  + df['SchoolVolunteers']

# average number of volunteers
ave_total_vols = df['TotalVolunteerNumber'].mean()
print(f"The average number of volunteers since 2016 is {round(ave_total_vols, 2)} people\n")

# average number of volunteers per year 
[print(f"The average number of volunteers in {x} is {round(df['TotalVolunteerNumber'].loc[df['ActivityDate'].dt.year == x].mean(), 2)}") for x in date_range]

# and what types of volunteers?
# No records of volunteers before 2019 therefor can exclude other years 
df_valid_volunteer = df.loc[df['ActivityDate'].dt.year == 2019]
print(f"\nThe average number of External Voluneers: {round(df_valid_volunteer['ExternalVolunteers'].mean(),2)}")
print(f"The average number of Hub Voluneers: {round(df_valid_volunteer['HubVolunteers'].mean(),2)}")
print(f"The average number of School Voluneers: {round(df_valid_volunteer['SchoolVolunteers'].mean(),2)}")

The average number of volunteers since 2016 is 0.16 people

The average number of volunteers in 2016 is 0.0
The average number of volunteers in 2017 is 0.0
The average number of volunteers in 2018 is 0.0
The average number of volunteers in 2019 is 0.61

The average number of External Voluneers: 0.24
The average number of Hub Voluneers: 0.2
The average number of School Voluneers: 0.17
